In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16223845697710117752, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5195939535686707531
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time
import math
import os
import pandas as pd
from keras.callbacks import EarlyStopping
import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize
from keras.utils.np_utils import to_categorical
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0


In [5]:
batch_size = 32
img_height, img_width = 128, 128
input_shape = (img_height, img_width, 3)
epochs = 15

In [6]:
train_dir = 'data/train/'
test_dir = 'data/test'

In [7]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split= 0.3,
    zoom_range=0.2,
    shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 34909 images belonging to 6 classes.
Found 14958 images belonging to 6 classes.
Found 13578 images belonging to 6 classes.


In [8]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 34909
nb_validation_samples: 14958
nb_test_samples: 13578

predict_size_train: 1091
predict_size_validation: 468
predict_size_test: 425

 num_classes: 6


In [9]:
# os.mkdir("extracted_features")
extracted_features_dir = "extracted_features/"
model_name = "Dense201_InceptionV3_descriptors"

In [10]:
# vgg19_weights ="../input/full-keras-pretrained-no-top/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_weights ="../input/full-keras-pretrained-no-top//inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
# vgg16_weights ="../input/full-keras-pretrained-no-top/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet201_weights ="../input/full-keras-pretrained-no-top/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet121_weights ="../input/full-keras-pretrained-no-top/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"
# resenet50_weights ="../input/full-keras-pretrained-no-top/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_resnet_v2_weights ="../input/full-keras-pretrained-no-top/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
# nasnet_weights ="../input/full-keras-pretrained-no-top/nasnet_large_no_top.h5"
# nasnet_mobile_weights ="../input/full-keras-pretrained-no-top/nasnet_mobile_no_top.h5"
# mobilenet_weights ="../input/full-keras-pretrained-no-top/mobilenet_1_0_224_tf_no_top.h5"

In [11]:
# from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
# from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception, preprocess_input
from keras.applications import DenseNet201
# from keras.applications import DenseNet121
# from keras.applications import ResNet50
# from keras.applications.inception_resnet_v2 import InceptionResNetV2
# from keras.applications import NASNetLarge, NASNetMobile
# from keras.applications import MobileNet

In [12]:
input_tensor = Input(shape = input_shape)  

base_model1=InceptionV3(input_shape= input_shape,weights='imagenet', include_top=False, input_tensor=input_tensor)
base_model2=DenseNet201(input_shape= input_shape,weights='imagenet', include_top=False, input_tensor=input_tensor)

x1 = base_model1.output
x1 = GlobalAveragePooling2D()(x1)

x2 = base_model2.output
x2 = GlobalAveragePooling2D()(x2)

merge = concatenate([x1, x2])
predictions = Dense(num_classes, activation='softmax')(merge)

model = Model(inputs=input_tensor,outputs=predictions)

In [13]:
for i, layer in enumerate(model.layers):
    print(i, layer.name)

0 input_1
1 zero_padding2d_1
2 conv1/conv
3 conv1/bn
4 conv1/relu
5 zero_padding2d_2
6 pool1
7 conv2_block1_0_bn
8 conv2_block1_0_relu
9 conv2_block1_1_conv
10 conv2_block1_1_bn
11 conv2_block1_1_relu
12 conv2_block1_2_conv
13 conv2_block1_concat
14 conv2_block2_0_bn
15 conv2_block2_0_relu
16 conv2_block2_1_conv
17 conv2_block2_1_bn
18 conv2_block2_1_relu
19 conv2_block2_2_conv
20 conv2_block2_concat
21 conv2_block3_0_bn
22 conv2_block3_0_relu
23 conv2_block3_1_conv
24 conv2_block3_1_bn
25 conv2_block3_1_relu
26 conv2_block3_2_conv
27 conv2_block3_concat
28 conv2_block4_0_bn
29 conv2_block4_0_relu
30 conv2_block4_1_conv
31 conv2_block4_1_bn
32 conv2_block4_1_relu
33 conv2_block4_2_conv
34 conv2_block4_concat
35 conv2_block5_0_bn
36 conv2_block5_0_relu
37 conv2_block5_1_conv
38 conv2_block5_1_bn
39 conv2_block5_1_relu
40 conv2_block5_2_conv
41 conv2_block5_concat
42 conv2_block6_0_bn
43 conv2_block6_0_relu
44 conv2_block6_1_conv
45 conv2_block6_1_bn
46 conv2_block6_1_relu
47 conv2_block

696 conv2d_27
697 conv2d_30
698 conv5_block18_1_relu
699 batch_normalization_27
700 batch_normalization_30
701 conv5_block18_2_conv
702 activation_27
703 activation_30
704 max_pooling2d_3
705 conv5_block18_concat
706 mixed3
707 conv5_block19_0_bn
708 conv2d_35
709 conv5_block19_0_relu
710 batch_normalization_35
711 conv5_block19_1_conv
712 activation_35
713 conv5_block19_1_bn
714 conv2d_36
715 conv5_block19_1_relu
716 batch_normalization_36
717 conv5_block19_2_conv
718 activation_36
719 conv5_block19_concat
720 conv2d_32
721 conv2d_37
722 conv5_block20_0_bn
723 batch_normalization_32
724 batch_normalization_37
725 conv5_block20_0_relu
726 activation_32
727 activation_37
728 conv5_block20_1_conv
729 conv2d_33
730 conv2d_38
731 conv5_block20_1_bn
732 batch_normalization_33
733 batch_normalization_38
734 conv5_block20_1_relu
735 activation_33
736 activation_38
737 average_pooling2d_4
738 conv5_block20_2_conv
739 conv2d_31
740 conv2d_34
741 conv2d_39
742 conv2d_40
743 conv5_block20_concat


In [14]:
c1 = model.layers[9].output 
c1 = GlobalAveragePooling2D()(c1)       

c2 = model.layers[55].output
c2 = GlobalAveragePooling2D()(c2)       

c3 = model.layers[143].output
c3 = GlobalAveragePooling2D()(c3)       

c4 = model.layers[483].output
c4 = GlobalAveragePooling2D()(c4) 

c5 = model.layers[571].output
c5 = GlobalAveragePooling2D()(c5) 

c6 = model.layers[585].output
c6 = GlobalAveragePooling2D()(c6) 

c7 = model.layers[601].output
c7 = GlobalAveragePooling2D()(c7) 

c8 = model.layers[634].output
c8 = GlobalAveragePooling2D()(c8) 

c9 = model.layers[677].output
c9 = GlobalAveragePooling2D()(c9) 

c10 = model.layers[708].output
c10 = GlobalAveragePooling2D()(c10) 

c11 = model.layers[739].output
c11 = GlobalAveragePooling2D()(c11) 

c12 = model.layers[787].output
c12 = GlobalAveragePooling2D()(c12) 

c13 = model.layers[835].output
c13 = GlobalAveragePooling2D()(c13) 

c14 = model.layers[883].output
c14 = GlobalAveragePooling2D()(c14) 

c15 = model.layers[933].output
c15 = GlobalAveragePooling2D()(c15) 

c16 = model.layers[954].output
c16 = GlobalAveragePooling2D()(c16) 

c17 = model.layers[996].output
c17 = GlobalAveragePooling2D()(c17) 

con = concatenate([c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14, c15, c16, c17])

bottleneck_final_model = Model(inputs=model.input, outputs=con)

In [15]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train, max_q_size=1, pickle_safe=False)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

In [16]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

In [17]:
bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [18]:

import tensorflow as tf
from keras.backend.tensorflow_backend import get_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import set_session

def reset_keras_tf_session():
    """
    this function clears the gpu memory and set the 
    tf session to not use the whole gpu
    """
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

#     config = tf.ConfigProto()
#     config.gpu_options.allow_growth = True
#     set_session(tf.Session(config=config))

reset_keras_tf_session()

In [ ]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [ ]:
space = {
         'lr': hp.choice('lr',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8, 0.9]),
         'beta_2':hp.choice('beta_2',[0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.999,0.99,0.995]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [ ]:
from sklearn.metrics import accuracy_score
def f_nn(params):   
    print ('Parameters testing: ', params)
  
    adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0)
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    model = Sequential()
    # model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.7))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_data, train_labels,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(validation_data, validation_labels),
                        verbose= 2,
                        callbacks=get_callbacks(params))

    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

    print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
    print("Validation Loss: {}".format(eval_loss))
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    preds = model.predict(test_data)

    predictions = [i.argmax() for i in preds]
    y_true = [i.argmax() for i in test_labels]
#     cm = confusion_matrix(y_pred=predictions, y_true=y_true)

    print('Test Accuracy: {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))

    print("*_*" * 50)
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': eval_loss, 'status': STATUS_OK, 'model': model}

In [ ]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    return callbacks

In [ ]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals = 300, trials=trials)
print(best)

Parameters testing:  {'beta_1': 0.7, 'lr': 1e-05, 'beta_2': 0.4}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 1.1467 - acc: 0.5954 - val_loss: 1.0632 - val_acc: 0.5486
Epoch 2/15
 - 4s - loss: 0.7226 - acc: 0.7529 - val_loss: 0.9028 - val_acc: 0.6209
Epoch 3/15
 - 4s - loss: 0.6164 - acc: 0.7868 - val_loss: 0.8242 - val_acc: 0.6652
Epoch 4/15
 - 4s - loss: 0.5575 - acc: 0.8102 - val_loss: 0.8021 - val_acc: 0.6706
Epoch 5/15
 - 4s - loss: 0.5231 - acc: 0.8196 - val_loss: 0.7596 - val_acc: 0.6974
Epoch 6/15
 - 4s - loss: 0.4950 - acc: 0.8301 - val_loss: 0.7410 - val_acc: 0.7047
Epoch 7/15
 - 4s - loss: 0.4726 - acc: 0.8375 - val_loss: 0.7271 - val_acc: 0.7111
Epoch 8/15
 - 4s - loss: 0.4532 - acc: 0.8443 - val_loss: 0.7345 - val_acc: 0.7090
Epoch 9/15
 - 4s - loss: 0.4381 - acc: 0.8483 - val_loss: 0.7294 - val_acc: 0.7097
Epoch 10/15
 - 4s - loss: 0.4262 - acc: 0.8531 - val_loss: 0.6985 - val_acc: 0.7267
Epoch 11/15
 - 4s - loss: 0.4117 - acc: 0.8580 - val_l

14958/14958 [==============================] - 1s 38us/step
Validation Accuracy: 59.1590%
Validation Loss: 0.9580243653290921
Test Accuracy: 0.7269848283988806
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.1, 'lr': 0.0001, 'beta_2': 0.3}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 0.7056 - acc: 0.7467 - val_loss: 0.7720 - val_acc: 0.6897
Epoch 2/15
 - 4s - loss: 0.4846 - acc: 0.8286 - val_loss: 0.7305 - val_acc: 0.7168
Epoch 3/15
 - 4s - loss: 0.4366 - acc: 0.8496 - val_loss: 0.8575 - val_acc: 0.7123
Epoch 4/15
 - 4s - loss: 0.4138 - acc: 0.8603 - val_loss: 0.7625 - val_acc: 0.7314
Epoch 5/15
 - 4s - loss: 0.4009 - acc: 0.8662 - val_loss: 0.7746 - val_acc: 0.7343
Epoch 6/15
 - 4s - loss: 0.3887 - acc: 0.8715 - val_loss: 0.7799 - val_acc: 0.7375
Epoch 7/15
 - 4s - loss: 0.3851 - acc: 0.8740 - val_loss: 0.7624 - val_acc:

Parameters testing:  {'beta_1': 0.6, 'lr': 0.1, 'beta_2': 0.5}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 12.3300 - acc: 0.2342 - val_loss: 12.4695 - val_acc: 0.2264
Epoch 2/15
 - 4s - loss: 12.3416 - acc: 0.2343 - val_loss: 12.6236 - val_acc: 0.2168
Epoch 3/15
 - 4s - loss: 12.1169 - acc: 0.2481 - val_loss: 12.6236 - val_acc: 0.2168
Epoch 4/15
 - 4s - loss: 11.9454 - acc: 0.2588 - val_loss: 12.6236 - val_acc: 0.2168
Epoch 00004: early stopping
14958/14958 [==============================] - 1s 39us/step
Validation Accuracy: 21.6807%
Validation Loss: 12.623578551938024
Test Accuracy: 0.2154956547356017
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.9, 'lr': 0.001, 'beta_2': 0.8}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 0.8570 - acc: 0.6661 - val_loss: 0.7212 - val_acc: 0.7185
Epoch 2/15

Epoch 2/15
 - 4s - loss: 0.7629 - acc: 0.7388 - val_loss: 0.9219 - val_acc: 0.6039
Epoch 3/15
 - 4s - loss: 0.6501 - acc: 0.7712 - val_loss: 0.8376 - val_acc: 0.6452
Epoch 4/15
 - 4s - loss: 0.5868 - acc: 0.7968 - val_loss: 0.8171 - val_acc: 0.6528
Epoch 5/15
 - 4s - loss: 0.5478 - acc: 0.8092 - val_loss: 0.7811 - val_acc: 0.6752
Epoch 6/15
 - 4s - loss: 0.5210 - acc: 0.8175 - val_loss: 0.7615 - val_acc: 0.6869
Epoch 7/15
 - 4s - loss: 0.4959 - acc: 0.8282 - val_loss: 0.7495 - val_acc: 0.6899
Epoch 8/15
 - 4s - loss: 0.4783 - acc: 0.8328 - val_loss: 0.7280 - val_acc: 0.7066
Epoch 9/15
 - 4s - loss: 0.4656 - acc: 0.8368 - val_loss: 0.7244 - val_acc: 0.7063
Epoch 10/15
 - 4s - loss: 0.4522 - acc: 0.8427 - val_loss: 0.7251 - val_acc: 0.7075
Epoch 11/15
 - 4s - loss: 0.4418 - acc: 0.8448 - val_loss: 0.7183 - val_acc: 0.7099
Epoch 12/15
 - 4s - loss: 0.4331 - acc: 0.8501 - val_loss: 0.7081 - val_acc: 0.7197
Epoch 13/15
 - 4s - loss: 0.4192 - acc: 0.8529 - val_loss: 0.7202 - val_acc: 0.7117


Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 6s - loss: 2.5609 - acc: 0.1736 - val_loss: 1.8159 - val_acc: 0.2218
Epoch 2/15
 - 4s - loss: 2.4042 - acc: 0.1841 - val_loss: 1.7463 - val_acc: 0.2017
Epoch 3/15
 - 4s - loss: 2.2829 - acc: 0.1966 - val_loss: 1.6979 - val_acc: 0.1874
Epoch 4/15
 - 4s - loss: 2.1862 - acc: 0.2134 - val_loss: 1.6634 - val_acc: 0.1885
Epoch 00004: early stopping
14958/14958 [==============================] - 1s 41us/step
Validation Accuracy: 18.8461%
Validation Loss: 1.6633873923861864
Test Accuracy: 0.2131389011636471
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.4, 'lr': 1e-05, 'beta_2': 0.4}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 1.2177 - acc: 0.5665 - val_loss: 1.0861 - val_acc: 0.5293
Epoch 2/15
 - 4s - loss: 0.7490 - acc: 0.7436 - val_loss: 0.9216 - val_acc: 0.610

...

Parameters testing:  {'beta_1': 0.8, 'lr': 0.0001, 'beta_2': 0.5}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 18s - loss: 0.6700 - acc: 0.7606 - val_loss: 0.7991 - val_acc: 0.6752
Epoch 2/15
 - 8s - loss: 0.4721 - acc: 0.8314 - val_loss: 0.7423 - val_acc: 0.7065
Epoch 3/15
 - 8s - loss: 0.4234 - acc: 0.8499 - val_loss: 0.6954 - val_acc: 0.7298
Epoch 4/15
 - 9s - loss: 0.4081 - acc: 0.8583 - val_loss: 0.7186 - val_acc: 0.7139
Epoch 5/15
 - 8s - loss: 0.3936 - acc: 0.8624 - val_loss: 0.6388 - val_acc: 0.7547
Epoch 6/15
 - 10s - loss: 0.3904 - acc: 0.8642 - val_loss: 0.6918 - val_acc: 0.7276
Epoch 7/15
 - 10s - loss: 0.3908 - acc: 0.8639 - val_loss: 0.6649 - val_acc: 0.7409
Epoch 8/15
 - 8s - loss: 0.3814 - acc: 0.8649 - val_loss: 0.7164 - val_acc: 0.7199
Epoch 00008: early stopping
14958/14958 [==============================] - 2s 115us/step
Validation Accuracy: 71.9882%
Validation Loss: 0.716441345636336
Test Accuracy: 0.8440123729562528
*_**_**_**_**_**_**_**_**_**_

 - 8s - loss: 1.7473 - acc: 0.3094 - val_loss: 1.5947 - val_acc: 0.3447
Epoch 14/15
 - 8s - loss: 1.7085 - acc: 0.3222 - val_loss: 1.5826 - val_acc: 0.3557
Epoch 15/15
 - 9s - loss: 1.6648 - acc: 0.3415 - val_loss: 1.5705 - val_acc: 0.3627
14958/14958 [==============================] - 2s 116us/step
Validation Accuracy: 36.2682%
Validation Loss: 1.570517840639189
Test Accuracy: 0.4987479746648991
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.9, 'lr': 0.0001, 'beta_2': 0.999}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 17s - loss: 0.6918 - acc: 0.7552 - val_loss: 0.7853 - val_acc: 0.6695
Epoch 2/15
 - 8s - loss: 0.4707 - acc: 0.8343 - val_loss: 0.6820 - val_acc: 0.7381
Epoch 3/15
 - 9s - loss: 0.4183 - acc: 0.8540 - val_loss: 0.6685 - val_acc: 0.7423
Epoch 4/15
 - 8s - loss: 0.3900 - acc: 0.8625 - val_loss: 0.6764 - val_acc: 0.731

Epoch 13/15
 - 8s - loss: 0.7742 - acc: 0.7332 - val_loss: 0.9912 - val_acc: 0.5743
Epoch 14/15
 - 8s - loss: 0.7529 - acc: 0.7414 - val_loss: 0.9756 - val_acc: 0.5802
Epoch 15/15
 - 8s - loss: 0.7328 - acc: 0.7490 - val_loss: 0.9613 - val_acc: 0.5869
14958/14958 [==============================] - 2s 105us/step
Validation Accuracy: 58.6910%
Validation Loss: 0.9612807979684476
Test Accuracy: 0.7244807777286787
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.2, 'lr': 0.0001, 'beta_2': 0.999}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 17s - loss: 0.6839 - acc: 0.7563 - val_loss: 0.7773 - val_acc: 0.6822
Epoch 2/15
 - 8s - loss: 0.4632 - acc: 0.8367 - val_loss: 0.7284 - val_acc: 0.7058
Epoch 3/15
 - 8s - loss: 0.4171 - acc: 0.8523 - val_loss: 0.7082 - val_acc: 0.7186
Epoch 4/15
 - 8s - loss: 0.3853 - acc: 0.8637 - val_loss: 0.6952 - v

Epoch 3/15
 - 9s - loss: 0.4117 - acc: 0.8543 - val_loss: 0.6812 - val_acc: 0.7343
Epoch 4/15
 - 8s - loss: 0.3817 - acc: 0.8652 - val_loss: 0.6432 - val_acc: 0.7565
Epoch 5/15
 - 8s - loss: 0.3609 - acc: 0.8716 - val_loss: 0.6512 - val_acc: 0.7448
Epoch 6/15
 - 9s - loss: 0.3489 - acc: 0.8758 - val_loss: 0.6966 - val_acc: 0.7179
Epoch 7/15
 - 9s - loss: 0.3360 - acc: 0.8809 - val_loss: 0.7481 - val_acc: 0.7163
Epoch 00007: early stopping
14958/14958 [==============================] - 2s 139us/step
Validation Accuracy: 71.6272%
Validation Loss: 0.7481428504554023
Test Accuracy: 0.8337015760789512
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.2, 'lr': 0.1, 'beta_2': 0.9}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 18s - loss: 12.3813 - acc: 0.2311 - val_loss: 12.3919 - val_acc: 0.2312
Epoch 2/15
 - 8s - loss: 12.3925 - acc: 0.2311

 - 18s - loss: 0.6702 - acc: 0.7591 - val_loss: 0.7259 - val_acc: 0.7084
Epoch 2/15
 - 8s - loss: 0.4875 - acc: 0.8281 - val_loss: 0.7332 - val_acc: 0.6966
Epoch 3/15
 - 8s - loss: 0.4538 - acc: 0.8402 - val_loss: 0.7114 - val_acc: 0.7145
Epoch 4/15
 - 8s - loss: 0.4362 - acc: 0.8460 - val_loss: 0.7318 - val_acc: 0.7137
Epoch 5/15
 - 8s - loss: 0.4304 - acc: 0.8477 - val_loss: 0.6884 - val_acc: 0.7308
Epoch 6/15
 - 8s - loss: 0.4244 - acc: 0.8518 - val_loss: 0.6867 - val_acc: 0.7363
Epoch 7/15
 - 8s - loss: 0.4301 - acc: 0.8506 - val_loss: 0.6921 - val_acc: 0.7410
Epoch 8/15
 - 9s - loss: 0.4264 - acc: 0.8496 - val_loss: 0.7626 - val_acc: 0.7249
Epoch 9/15
 - 9s - loss: 0.4333 - acc: 0.8458 - val_loss: 0.7139 - val_acc: 0.7432
Epoch 10/15
 - 8s - loss: 0.4390 - acc: 0.8450 - val_loss: 0.6918 - val_acc: 0.7489
Epoch 11/15
 - 8s - loss: 0.4307 - acc: 0.8455 - val_loss: 0.7688 - val_acc: 0.7210
Epoch 12/15
 - 8s - loss: 0.4414 - acc: 0.8458 - val_loss: 0.6893 - val_acc: 0.7626
Epoch 13/15